In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
from glob import glob
import os#, sys

import matplotlib.pyplot as plt

import rioxarray as rio
from shapely.geometry import mapping

%matplotlib inline
# %matplotlib widget

In [ ]:
### every year-hourly data to resample to annual resolution (sum of precipitation) 

In [ ]:
# read here about why its so slow:
# https://stackoverflow.com/questions/45039794/wrong-time-dimension-after-doing-a-groupby-with-library-xarray-python

In [6]:
# %%time

# READ:

# temps = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/2m-temperature/*.nc")
# precips = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/*.nc")#, chunks={"time": 10000})

path = "/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/"
for_output = "/Users/varyabazilova/Desktop/hma_regions/era5_resampled_to_annual"

In [1]:
# iterate over every year-long file -> resample to annual, append to a new xr.Dataset -> new dataset with annual data: 

In [18]:
%%time

annual = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # group by -> resample to annual 
    ds_sum = ds.groupby('time.year').sum('time')
    print(ds_sum)
    # merge 
    annual = xr.merge([annual, ds_sum])

# print(annual)

<xarray.Dataset>
Dimensions:    (longitude: 277, latitude: 197, year: 1)
Coordinates:
  * longitude  (longitude) float32 55.0 55.25 55.5 55.75 ... 123.5 123.8 124.0
  * latitude   (latitude) float32 55.0 54.75 54.5 54.25 ... 6.75 6.5 6.25 6.0
  * year       (year) int64 1979
Data variables:
    tp         (year, latitude, longitude) float32 0.5732 0.5742 ... 2.186 1.916
<xarray.Dataset>
Dimensions:    (longitude: 277, latitude: 197, year: 1)
Coordinates:
  * longitude  (longitude) float32 55.0 55.25 55.5 55.75 ... 123.5 123.8 124.0
  * latitude   (latitude) float32 55.0 54.75 54.5 54.25 ... 6.75 6.5 6.25 6.0
  * year       (year) int64 1980
Data variables:
    tp         (year, latitude, longitude) float32 0.6375 0.6409 ... 2.539 2.249
<xarray.Dataset>
Dimensions:    (longitude: 277, latitude: 197, year: 1)
Coordinates:
  * longitude  (longitude) float32 55.0 55.25 55.5 55.75 ... 123.5 123.8 124.0
  * latitude   (latitude) float32 55.0 54.75 54.5 54.25 ... 6.75 6.5 6.25 6.0
  * year   

In [60]:

# annual = annual.drop('expver') # uncomment this if you do it again


In [58]:
annual.tp.values.max()

14.624654

In [63]:
annual.attrs['units'] = 'm'
annual.attrs['method'] = 'annual sum from raw hourly era5 data'

In [70]:
# annual save to netcdf: 

# annual.to_netcdf("/Users/varyabazilova/Desktop/hma_regions/era5_resampled_to_annual/total_precipitation_annual_sum_1979to2020.nc")